In [2]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.Collecting pandas
  Using cached pandas-2.2.2-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp311-cp311-win_amd64.whl (11.6 MB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.2.28 requires opencv-python>=4.6.0, which is not installed.
ultralytics 8.2.28 requires py-cpuinfo, which is not installed.
ultralytics 8.2.28 requires seaborn>=0.11.0, which is not installed.
ultralytics 8.2.28 requires tqdm>=4.64.0, which is not installed.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

: 

In [ ]:
model_name = "lstmv1PLNtoUAH"

In [ ]:
# Wczytanie danych z pliku CSV
# Upewnij się, że ścieżka do pliku jest poprawna
input_data = 'plnuah_d.csv'
df = pd.read_csv(input_data)
# Tworzenie X (liczba porządkowa) i y (wartość otwarcia)
X = np.arange(len(df)).reshape(-1, 1)
y = df['Otwarcie'].values
# Wyświetlenie danych
plt.plot(X, y, color='black', label='Data')
plt.xlabel('dni od 2014-05-19 do 2024-05-18')
plt.ylabel('Wartość otwarcia w dniu EURO w PLN')
plt.title('10 lat - EURO w PLN')
plt.savefig(f'data_{model_name}.png')
plt.show()
# Przeskalowanie wartości y do zakresu od 0 do 1
min_y = np.min(y)
max_y = np.max(y)
y = (y - min_y) / (max_y - min_y)

In [ ]:
# Funkcja do przygotowania sekwencji danych
def create_sequences(y_data, seq_length):
    X_sequences = []
    y_labels = []
    for i in range(len(y_data) - seq_length):
        X_sequences.append(y_data[i:i+seq_length])
        y_labels.append(y_data[i+seq_length])
    return np.array(X_sequences), np.array(y_labels)

In [ ]:
# Długość sekwencji
seq_length = 50

In [ ]:
# Tworzenie sekwencji
X_seq, y_seq = create_sequences(y, seq_length)
print(f'Kształt X_seq: {X_seq.shape}, Kształt y_seq: {y_seq.shape}')

In [ ]:
# Podział danych na zbiór treningowy i testowy
split_index = round(0.8 * len(X_seq))
X_train, X_test = X_seq[:split_index], X_seq[split_index:]
y_train, y_test = y_seq[:split_index], y_seq[split_index:]
print(f'Kształt X_train: {X_train.shape}, Kształt y_train: {y_train.shape}')
print(f'Kształt X_test: {X_test.shape}, Kształt y_test: {y_test.shape}')

In [ ]:
# Definicja modelu
Dropout_rate=0.2
model = Sequential()
'''do uzupełnienia'''
model.add(Dense(1))  # Warstwa wyjściowa z jednym neuronem

In [ ]:
# Kompilacja modelu
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Trenowanie modelu
batch_size = round(len(X_train)/10)
print('batch_size =',batch_size)
history = model.fit(X_train, y_train, epochs=100, batch_size=batch_size, verbose=1)

In [ ]:
# Plotting Loss vs Epochs
plt.figure(figsize =(10,7))
plt.plot(history.history["loss"])
plt.xlabel("Epochs")
plt.ylabel("Losses")
plt.title("RNN model, Loss MSE vs Epoch")
plt.ylim(0, 0.1)  # Ustawienie zakresu osi y od 0 do 0.1
mid_x = len(history.history["loss"]) / 2  # Środek osi x (liczba epok podzielona przez 2)
mid_y = 0.05  # Środek osi y (0.05 to połowa zakresu od 0 do 0.1)
plt.text(mid_x, mid_y, f' Sequence length = {seq_length} \n Batch size={batch_size} \n Dropout rate={Dropout_rate} \n Input data - {input_data} \n Final MSE = {round(history.history["loss"][-1],5)}', fontsize=16, ha='left')# Dodanie tekstu na środku obrazka
plt.savefig(f'history_{model_name}.png')
plt.show()

In [ ]:
# Przewidywania na danych treningowych
predictions_train = model.predict(X_train)
# wykres
plt.plot(predictions_train.flatten(), color='red', label='Predykcje')
plt.plot(y_train.flatten(), color='blue', label='Dane treningowe',alpha=0.5)
plt.legend()
mid_x = len(predictions_train) / 2  # Środek osi x (liczba epok podzielona przez 2)
mid_y = 1/10  # Środek osi y (0.05 to połowa zakresu od 0 do 0.1)
#Residual Sum of Squares
plt.text(mid_x, mid_y, f' RSS = {round(np.sum((predictions_train.flatten() - y_train.flatten()) ** 2),4)} \n MSE = {round(np.mean((predictions_train.flatten() - y_train.flatten()) ** 2),6)} ', fontsize=16, ha='left')# Dodanie tekstu na środku obrazka
plt.savefig(f'pred_train_{model_name}.png')
plt.show()

In [ ]:
# Przewidywania na danych testowch
predictions_test = model.predict(X_test)
# wykres
plt.plot(predictions_test.flatten(), color='red', label='Predykcje',alpha=1)
plt.plot(y_test.flatten(), color='black', label='Dane testowe',alpha=0.5)
plt.legend()
mid_x = len(predictions_test) / 8  # Środek osi x (liczba epok podzielona przez 2)
mid_y = 10/35  # Środek osi y (0.05 to połowa zakresu od 0 do 0.1)
#Residual Sum of Squares
plt.text(mid_x, mid_y, f' RSS = {round(np.sum((predictions_test.flatten() - y_test.flatten()) ** 2),4)} \n MSE = {round(np.mean((predictions_test.flatten() - y_test.flatten()) ** 2),6)} ', fontsize=16, ha='left')# Dodanie tekstu na środku obrazka
plt.savefig(f'pred_test_{model_name}.png')
plt.show()

In [ ]:
# export modelu RNN
model.save(f"keras_{model_name}.keras")

In [ ]:
# Wizualizacja modelu
from keras.utils import plot_model
plot_model(model, to_file=f'architecture_{model_name}.png', show_shapes=True, show_layer_names=True)